In [18]:
import bs4 as bs
import re
import json
from math import ceil
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

# Extracting categories and their links 

In [19]:
site= "https://www.daraz.pk/"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(site,headers=hdr)
page = urlopen(req)
data = BeautifulSoup(page, 'html.parser')
print(data)


<!DOCTYPE HTML>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="a2a0e.home" name="spm-id"/>
<meta content="" name="keywords"/>
<meta content="Pakistan's best online shopping store with 15+ million products at resounding discounts in Karachi ✓ Lahore ✓ Islamabad ✓ All across Pakistan with cash on delivery (COD). Pick your favorite Mobiles, Appliances, Apparels, and Fashion accessories on amazing deals exclusively available at Daraz.pk." name="description"/>
<meta content="https://www.daraz.pk/" property="og:url">
<meta content="website" property="og:type"/>
<meta content="Online Shopping in Pakistan: Fashion, Electronics &amp; Books - Daraz.pk" property="og:title">
<meta content="Pakistan's best online shopping store with 15+ million products at resounding discounts in Karachi ✓ Lahore ✓ Islamabad ✓ All across Pakistan with cash on delivery (COD). Pick your favorite Mobiles, Appliances, Apparels, and Fashion accessories on amazing deals exclusively available at Daraz.pk

In [ ]:
x = data.find('ul', attrs={'class', 'lzd-site-menu-root'})
x = x.find_all("a")
x = x[12:]
category_list = []
for y in x:
    categories = {}
    categories["category name"] = y.text[1:-1]
    test = y.get('href')
    url = 'https:'+str(test)
    categories["category link"] = url
    
    driver = webdriver.Chrome('chromedriver')
    driver.get(url)
    total_products_category_element = driver.find_element(By.XPATH, "//div[@class='result-tips--Knf5Y']/div/div/span")
    total_products_category = [int(i) for i in total_products_category_element.text.split() if i.isdigit()]
    driver.quit()
    categories["total products"] = total_products_category[0]
    
    
    category_list.append(categories)
print(category_list)

In [252]:
dataFrame = pd.DataFrame.from_dict(category_list)
print(dataFrame)

dataFrame.to_csv('categories data.csv', index=False)

                 category name  \
0                    Beverages   
1               Fresh Products   
2    Breakfast, Choco & Snacks   
3                 Food Staples   
4              Dairy & Chilled   
..                         ...   
118        Women's Accessories   
119       Sunglasses & Eyewear   
120                 Automotive   
121                 Motorcycle   
122         Loaders & Rickshaw   

                                         category link  total products  
0                      https://www.daraz.pk/beverages/           16176  
1              https://www.daraz.pk/buy-fresh-produce/             382  
2                      https://www.daraz.pk/breakfast/            7123  
3    https://www.daraz.pk/baking-and-cooking/?spm=a...           45487  
4        https://www.daraz.pk/groceries-dairy-chilled/            6533  
..                                                 ...             ...  
118  https://www.daraz.pk/womens-accessories/?spm=a...          335737  
119    

# Extracting all pages links of all categories

In [16]:
def get_pages_list(category_url, total_number_of_products):
    '''
    params: category_url : url of the category
            total_number_of_products : total number of products for that category 

    returns:    1. method computes the possible number of pages according to the number of products
                2. Creates a new list of url for all the page and then returns the list
    '''

    total_pages_category = ceil(total_number_of_products / 40)

    if total_pages_category > 1:
        category_pages_list = []

        for page in range(1, total_pages_category+1):
            if page == 1:
                category_pages_list.append(category_url)
            elif page >= 105:
                break
            else:
                category_pages_list.append(category_url+'?page='+str(page))
        return category_pages_list
    else:
        return []

In [17]:
import csv
 
# opening the CSV file
with open('categories data.csv', mode ='r')as file:
    
  # reading the CSV file
    csvFile = csv.reader(file)
 
  # displaying the contents of the CSV file
    counter = 0
    links_data = []
    for lines in csvFile:
        counter +=1
        if counter == 1:
            continue
        else:
            pages_list = get_pages_list(lines[1],int(lines[2]))
            for p in pages_list:
                details = {}
                details["category name"] = lines[0]
                details["page link"] = p
#                 print(details["category name"])
#                 print(details["page link"])
                links_data.append(details)
# print(links_data)
pageslink_data = pd.DataFrame.from_dict(links_data)
# print(pageslink_data)
pageslink_data.to_csv('pageslink_data.csv', index=False)

# Extracting Products links

In [6]:
import csv
page_links = []
with open('pageslink_data.csv', mode ='r')as f:
    myFile = csv.reader(f)
    counter = 0
    for rows in myFile:
        counter +=1
        if counter == 1:
            continue
        else:
#             print(rows[1])
            page_links.append(rows[1])
    
#     print(page_links)   

In [7]:
products_list = []
for page in page_links:

    html_paged = webdriver.Chrome('chromedriver')
    html_paged.get(page)

    if html_paged:
        for i in range(1, 40):
            '''
            Reason for iterating 40 elements is that Daraz Only Displays 40 items on its page, Hence we will fetch the product url for 40 items and then restart the counter for the next page.
            '''
            try:
                target_name = html_paged.find_element("xpath",'//*[@class="ant-col-20 ant-col-push-4 side-right--Tyehf"]/div[2]/div['+str(i)+']/div/div/div[2]/div[2]/a')
                if target_name:
                    pr_link = target_name.get_attribute("href")
                    products_list.append(pr_link)
                    print (pr_link)
                    # These product links would be written to an excel file and then next script will take care to cal the fetchInfo() method
                else:
                    break
            except:
                break
    else:
        continue
    html_paged.quit()

https://www.daraz.pk/products/kashmiri-tea-kashmiri-chaye-pink-tea-for-10ltr-milk-50-cup-i194684527-s1907697985.html?search=1
https://www.daraz.pk/products/family-mixture-430gm-i3849188-s14616742.html?search=1
https://www.daraz.pk/products/t-tox-i176650558-s1351248373.html?search=1
https://www.daraz.pk/products/danedar-430gm-i3849185-s14616739.html?search=1
https://www.daraz.pk/products/tezdum-430gm-i3803407-s14435441.html?search=1
https://www.daraz.pk/products/100-i222972949-s1439958631.html?search=1
https://www.daraz.pk/products/uuu-uu-uu-u-u-uu-u-u-i157046935-s1323290296.html?search=1
https://www.daraz.pk/products/tezdum-170gm-i3803404-s14435431.html?search=1
https://www.daraz.pk/products/danedar-900gm-i3849186-s14616740.html?search=1
https://www.daraz.pk/products/klassno-cappuccino-vero-gusto-with-choco-sprinkles-imported-coffee-10-sachet-full-box-i372066072-s1854923198.html?search=1
https://www.daraz.pk/products/30-i135480735-s1295732485.html?search=1
https://www.daraz.pk/products

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x010BF243]
	(No symbol) [0x01047FD1]
	(No symbol) [0x00F3D04D]
	(No symbol) [0x00F22D7A]
	(No symbol) [0x00F8BE7B]
	(No symbol) [0x00F9C196]
	(No symbol) [0x00F88386]
	(No symbol) [0x00F6163C]
	(No symbol) [0x00F6269D]
	GetHandleVerifier [0x01359A22+2655074]
	GetHandleVerifier [0x0134CA24+2601828]
	GetHandleVerifier [0x01168C0A+619850]
	GetHandleVerifier [0x01167830+614768]
	(No symbol) [0x010505FC]
	(No symbol) [0x01055968]
	(No symbol) [0x01055A55]
	(No symbol) [0x0106051B]
	BaseThreadInitThunk [0x755CFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77027BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77027B8E+238]


In [8]:
print(products_list)

['https://www.daraz.pk/products/kashmiri-tea-kashmiri-chaye-pink-tea-for-10ltr-milk-50-cup-i194684527-s1907697985.html?search=1', 'https://www.daraz.pk/products/family-mixture-430gm-i3849188-s14616742.html?search=1', 'https://www.daraz.pk/products/t-tox-i176650558-s1351248373.html?search=1', 'https://www.daraz.pk/products/danedar-430gm-i3849185-s14616739.html?search=1', 'https://www.daraz.pk/products/tezdum-430gm-i3803407-s14435441.html?search=1', 'https://www.daraz.pk/products/100-i222972949-s1439958631.html?search=1', 'https://www.daraz.pk/products/uuu-uu-uu-u-u-uu-u-u-i157046935-s1323290296.html?search=1', 'https://www.daraz.pk/products/tezdum-170gm-i3803404-s14435431.html?search=1', 'https://www.daraz.pk/products/danedar-900gm-i3849186-s14616740.html?search=1', 'https://www.daraz.pk/products/klassno-cappuccino-vero-gusto-with-choco-sprinkles-imported-coffee-10-sachet-full-box-i372066072-s1854923198.html?search=1', 'https://www.daraz.pk/products/30-i135480735-s1295732485.html?search

In [12]:
products_links = pd.DataFrame.from_dict(products_list)
products_links.to_csv('products_links.csv', index=False)

# Fetching product details

In [2]:
import csv
product_links = []
with open('products_links.csv', mode ='r')as t:
    li = csv.reader(t)
    counter = 0
    for rows in li:
        counter +=1
        if counter == 1:
            continue
        else:
#             print(rows[0])
            product_links.append(rows[0])

print(product_links)

['https://www.daraz.pk/products/complan-chocolate-flavour-bottle-400g-i208550191-s1413978329.html?search=1', 'https://www.daraz.pk/products/kashmiri-tea-kashmiri-chaye-pink-tea-for-10ltr-milk-50-cup-i194684527-s1907697985.html?search=1', 'https://www.daraz.pk/products/lipton-yellow-label-black-tea-danedar-90gm-i170862691-s1341128570.html?search=1', 'https://www.daraz.pk/products/family-mixture-430gm-i3849188-s14616742.html?search=1', 'https://www.daraz.pk/products/brooke-bond-supreme-black-tea-170gm-i3845454-s14602737.html?search=1', 'https://www.daraz.pk/products/t-tox-i176650558-s1351248373.html?search=1', 'https://www.daraz.pk/products/100-i222972949-s1439958631.html?search=1', 'https://www.daraz.pk/products/danedar-430gm-i3849185-s14616739.html?search=1', 'https://www.daraz.pk/products/tezdum-430gm-i3803407-s14435441.html?search=1', 'https://www.daraz.pk/products/brooke-bond-supreme-black-tea-430gm-i142190953-s1302450649.html?search=1', 'https://www.daraz.pk/products/uuu-uu-uu-u-u-

In [3]:
def fetchInfo(product_url):
    '''
    params: product_url : link of the product

    returns: basic information of product 
    '''

    html_doc = requests.get(product_url).text
    data = re.search(r'pdpTrackingData = ("{.*}")', html_doc)

    if data is None:
        return {"status": False, "error": "Failed to retrieve product information"}

    data = json.loads(data.group(1))
    data = data.replace('\"', '"')
    data = json.loads(data)

    result = {}
    result['name'] = data['pdt_name']
    result['seller'] = data['seller_name']
    result['price'] = data['pdt_price']
    result['image'] = data['pdt_photo']
    result['category'] = data['pdt_category']
    result['discount'] = data['pdt_discount']
    result['brand_name'] = data['brand_name']
    result['pdt_sku'] = data['pdt_sku']
    result['pdt_simplesku'] = data['pdt_simplesku']
    result['country'] = data['core']['country']
    return result

In [4]:
product_detail = []
for link in product_links:
    product_detail.append(fetchInfo(link))
    print(fetchInfo(link))

{'name': 'Complan Chocolate Flavour, Bottle, 400g', 'seller': 'Word of Taste', 'price': 'Rs. 1,349', 'image': 'https://static-01.daraz.pk/p/ff02a6a50ece30291044f436421a86a7.jpg', 'category': ['Groceries', 'Beverages', 'Nutrition Drinks'], 'discount': '-4%', 'brand_name': 'Complan', 'pdt_sku': 208550191, 'pdt_simplesku': 1413978329, 'country': 'PK'}
{'name': 'Kashmiri Tea - Kashmiri chaye - Pink Tea (For 10Ltr Milk-50 Cup)', 'seller': '7 Herbal', 'price': 'Rs. 190', 'image': 'https://pk-live-21.slatic.net/kf/Sba7f4d164a714a138e8548ecd265d38ex.jpg', 'category': ['Groceries', 'Beverages', 'Green Tea'], 'discount': '-16%', 'brand_name': 'No Brand', 'pdt_sku': 194684527, 'pdt_simplesku': 1907697985, 'country': 'PK'}
{'name': 'Lipton Yellow Label Black Tea Danedar 90gm', 'seller': 'Lipton', 'price': 'Rs. 150', 'image': 'https://static-01.daraz.pk/p/0fe8a69db0ac11629b32d97e60a304d9.jpg', 'category': ['Groceries', 'Beverages', 'Black Tea'], 'discount': '-8%', 'brand_name': 'Lipton', 'pdt_sku':

AttributeError: type object 'DataFrame' has no attribute 'from_dic'

In [5]:
products_details = pd.DataFrame.from_dict(product_detail)
products_details.to_csv('products_details.csv', index=False)